In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from ct import Ct
from viz import VolumePlot

DEVICE = 'cuda:0'

In [ ]:
best_model = torch.load('./results/heart_segmentation_2/best_model_fpn.pth', map_location=DEVICE)

In [ ]:
ct = Ct('/data/calcium_processed/CAC_011/ct/', file_pattern='IM-0001-*.dcm')

In [ ]:
masked = torch.empty(ct.img.shape)
mask = torch.empty(ct.img.shape)

for i in range(0, ct.img.shape[0]):
    image = ct.img[i] / 1000
    mask[i] = best_model.predict(image.to(DEVICE).unsqueeze(0).unsqueeze(0)).to('cpu') > 0.1
    masked[i] = image * (mask[i]<0.1) + (mask[i]*2)

In [ ]:
crops = [{ 'x':0, 'y':0, 'w':0, 'h':0 }] * mask.shape[0]

min_z = mask.shape[0]
max_z = 0

min_y = mask.shape[1]
max_y = 0

min_x = mask.shape[2]
max_x = 0

for i in range(0, mask.shape[0]):
    mask_slice = mask[i]
    mask_np = mask_slice.numpy().astype(np.uint8)

    contours, hierarchy = cv2.findContours(mask_np,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

    # Find object with the biggest bounding box
    mx = (0,0,0,0)      # biggest bounding box so far
    mx_area = 0
    for cont in contours:
        x,y,w,h = cv2.boundingRect(cont)
        area = w*h
        if area > mx_area:
            mx = x,y,w,h
            mx_area = area
    x,y,w,h = mx
    # rectangle is big enough
    if w > 20 and h > 20:
        if min_z > i: min_z = i
        if max_z < i: max_z = i
        if min_x > x: min_x = x
        if max_x < x+w: max_x = x+w
        if min_y > y: min_y = y
        if max_y < y+h: max_y = y+h
        crops[i] = { 'x':x, 'y':y, 'w':w, 'h':h }

    print(crops[i]['x'], crops[i]['y'], crops[i]['w'], crops[i]['h'])

min_z, max_z, min_y, max_y, min_x, max_x

In [ ]:
%matplotlib widget
cropped_volume = ct.img[min_z:max_z,min_y:max_y,min_x:max_x]

plotter = VolumePlot(cropped_volume, figsize=(9,9))
plotter.plot()

masked.shape, cropped_volume.shape

In [ ]:
plotter.plot_interactive()

In [ ]:
import pickle

with open('./results/heart_crop_1/CAC_001.pkl', 'rb') as file:
    bounding_volume = pickle.load(file)

ct = Ct('/data/calcium_processed/CAC_001/ct/', file_pattern='IM-0001-*.dcm')

In [ ]:
crop = ct.img[bounding_volume['min_z']:bounding_volume['max_z'],
              bounding_volume['min_y']:bounding_volume['max_y'],
              bounding_volume['min_x']:bounding_volume['max_x']]

plotter = VolumePlot(crop, figsize=(9,9))
plotter.plot()

crop.shape, crop.dtype